In [1]:
import os

# Ganti 'nama_folder' dengan nama folder yang ingin Anda masuki
path = 'suara'

# Gunakan metode `chdir` untuk berpindah ke folder tersebut
os.chdir(path)

# Sekarang Anda berada di dalam folder tersebut

In [2]:
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_Sad','OAF_Pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_Fear','OAF_disgust',
         'OAF_angry',
         ]

In [3]:
# Import Library
import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI STATISTIKA
    mean = np.mean(y)
    std_dev = np.std(y)
    max_value = np.max(y)
    min_value = np.min(y)
    median = np.median(y)
    skewness = skew(y)  # Calculate skewness
    kurt = kurtosis(y)  # Calculate kurtosis
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    mode_value, _ = mode(y)  # Calculate mode
    iqr = q3 - q1

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])



    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew]

In [5]:
features =[]

In [6]:
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            features.append([folder, filename] + statistics)

KeyboardInterrupt: 

In [7]:
# Membuat DataFrame dari data
columns =  ['Label', 'File'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew']
df = pd.DataFrame(features, columns=columns)

In [8]:
# Menampilkan file CSV
df

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew
0,YAF_sad,YAF_back_sad.wav,0.128581,0.033203,0.225249,3.231511,2.213080
1,YAF_sad,YAF_bar_sad.wav,0.111230,0.036133,0.196220,5.189765,2.569729
2,YAF_sad,YAF_base_sad.wav,0.196274,0.042480,0.259866,-0.207301,1.226778
3,YAF_sad,YAF_bath_sad.wav,0.145458,0.039062,0.222568,2.644984,1.998682
4,YAF_sad,YAF_bean_sad.wav,0.105494,0.035156,0.194269,5.715395,2.682275
...,...,...,...,...,...,...,...
2795,OAF_angry,OAF_witch_angry.wav,0.095215,0.070801,0.084418,2.776079,1.733152
2796,OAF_angry,OAF_yearn_angry.wav,0.083991,0.051270,0.091742,7.208814,2.744011
2797,OAF_angry,OAF_yes_angry.wav,0.115001,0.060059,0.118703,3.370580,2.012270
2798,OAF_angry,OAF_young_angry.wav,0.085005,0.059082,0.090037,5.366187,2.422199


In [9]:
df.to_csv('emosi_5_fitur.csv',index=False)

In [10]:
from sklearn.preprocessing import StandardScaler
# Baca file CSV
dn = pd.read_csv("emosi_5_fitur.csv")
# Tentukan kolom yang akan distandarisasi
kolom = ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew']
# Inisialisasi StandardScaler
scaler = StandardScaler()
# Lakukan standarisasi pada kolom yang telah ditentukan
dn[kolom] = scaler.fit_transform(dn[kolom])
# Simpan DataFrame yang telah distandarisasi ke dalam file CSV baru
dn.to_csv("emosi2normalisasi.csv", index=False)


In [11]:
norm=pd.read_csv('emosi2normalisasi.csv')
norm

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew
0,YAF_sad,YAF_back_sad.wav,-0.321317,-1.052521,0.940845,0.204442,0.495211
1,YAF_sad,YAF_bar_sad.wav,-0.643634,-0.962423,0.324040,1.185171,1.219225
2,YAF_sad,YAF_base_sad.wav,0.936230,-0.767212,1.676354,-1.517779,-1.507027
3,YAF_sad,YAF_bath_sad.wav,-0.007778,-0.872326,0.883874,-0.089302,0.059975
4,YAF_sad,YAF_bean_sad.wav,-0.750198,-0.992456,0.282586,1.448416,1.447698
...,...,...,...,...,...,...,...
2795,OAF_angry,OAF_witch_angry.wav,-0.941158,0.103730,-2.051468,-0.023647,-0.479064
2796,OAF_angry,OAF_yearn_angry.wav,-1.149659,-0.496920,-1.895834,2.196348,1.573026
2797,OAF_angry,OAF_yes_angry.wav,-0.573586,-0.226627,-1.322985,0.274090,0.087558
2798,OAF_angry,OAF_young_angry.wav,-1.130833,-0.256660,-1.932072,1.273526,0.919734


In [12]:
# Daftar kolom yang ingin dilewati
kolomlabel= ['Label','File',]
# Menghitung rata-rata untuk kolom numerik tertentu (mengabaikan kolom yang tidak diinginkan)
rata2= norm.drop(columns=kolomlabel).mean()
#membulatkan hasil komputasi dengan round dengan ketentuan 2 setelah koma, biar ga panjang bestiiiiiii
dibulatkan=rata2.round(2)
# Menampilkan rata-rata
print('--MEAN--')
print(dibulatkan)


--MEAN--
ZCR Mean        0.0
ZCR Median      0.0
ZCR Std Dev    -0.0
ZCR Kurtosis    0.0
ZCR Skew       -0.0
dtype: float64


In [13]:
# Daftar kolom yang ingin dilewati
kolomlabel= ['Label','File']
# Menghitung rata-rata untuk kolom numerik tertentu (mengabaikan kolom yang tidak diinginkan)
standv= norm.drop(columns=kolomlabel).std()
#membulatkan hasil komputasi dengan round dengan ketentuan 2 setelah koma, biar ga panjang bestiiiiiii
bulatkan=standv.round(2)
# Menampilkan rata-rata
print('--STANDARD DEVIASI--')
print(bulatkan)


--STANDARD DEVIASI--
ZCR Mean        1.0
ZCR Median      1.0
ZCR Std Dev     1.0
ZCR Kurtosis    1.0
ZCR Skew        1.0
dtype: float64


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [15]:
# Baca data dari file CSV
dataknn= pd.read_csv('emosi2normalisasi.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label','File'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
print(X)


      ZCR Mean  ZCR Median  ZCR Std Dev  ZCR Kurtosis  ZCR Skew
0    -0.321317   -1.052521     0.940845      0.204442  0.495211
1    -0.643634   -0.962423     0.324040      1.185171  1.219225
2     0.936230   -0.767212     1.676354     -1.517779 -1.507027
3    -0.007778   -0.872326     0.883874     -0.089302  0.059975
4    -0.750198   -0.992456     0.282586      1.448416  1.447698
...        ...         ...          ...           ...       ...
2795 -0.941158    0.103730    -2.051468     -0.023647 -0.479064
2796 -1.149659   -0.496920    -1.895834      2.196348  1.573026
2797 -0.573586   -0.226627    -1.322985      0.274090  0.087558
2798 -1.130833   -0.256660    -1.932072      1.273526  0.919734
2799 -1.324499   -0.587017    -1.750587      2.491151  1.906111

[2800 rows x 5 columns]


In [16]:
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler
scaler = StandardScaler()
# fit scaler on the training dataset
scaler.fit(X_train)
# save the scaler
dump(scaler, open('scaler5_fitur.pkl', 'wb'))
# transform the training dataset
X_train_scaled = scaler.transform(X_train)

In [37]:
from sklearn.decomposition import PCA

# Membuat objek PCA dengan jumlah komponen yang diinginkan
pca = PCA(n_components=2)  # Ganti 2 dengan jumlah komponen yang Anda inginkan

# Fit dan transformasikan data latih yang telah diubah skala
X_train_pca = pca.fit_transform(X_train_scaled)

# Transformasikan juga data uji menggunakan scaler yang sama
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

# X_train_pca sekarang berisi data latih dengan dimensi yang sudah direduksi
# X_test_pca sekarang berisi data uji dengan dimensi yang sudah direduksi


In [38]:
import pickle
with open('scaler5_fitur.pkl', 'rb') as standarisasi:
    loadscal= pickle.load(standarisasi)

In [39]:
X_test_scaled=loadscal.transform(X_test) #normalisasi X testing dari hasil normalisasi X train yang disimpan dalam model

In [40]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [41]:
best_accuracy = 0
best_k = 0

# Perulangan untuk mencari akurasi tertinggi
for n in range(1, 30):  # Ganti dengan rentang nilai k yang ingin Anda uji
    knn = KNeighborsClassifier(n_neighbors=n, metric='euclidean')
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Akurasi dengan k={n} adalah {accuracy}')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = n

print(f'Akurasi terbaik adalah {best_accuracy} dengan nilai k={best_k}')

Akurasi dengan k=1 adalah 0.45
Akurasi dengan k=2 adalah 0.4392857142857143
Akurasi dengan k=3 adalah 0.4607142857142857
Akurasi dengan k=4 adalah 0.49642857142857144
Akurasi dengan k=5 adalah 0.48214285714285715
Akurasi dengan k=6 adalah 0.49642857142857144
Akurasi dengan k=7 adalah 0.4857142857142857
Akurasi dengan k=8 adalah 0.48214285714285715
Akurasi dengan k=9 adalah 0.4857142857142857
Akurasi dengan k=10 adalah 0.48214285714285715
Akurasi dengan k=11 adalah 0.48392857142857143
Akurasi dengan k=12 adalah 0.475
Akurasi dengan k=13 adalah 0.475
Akurasi dengan k=14 adalah 0.4875
Akurasi dengan k=15 adalah 0.4928571428571429
Akurasi dengan k=16 adalah 0.5125
Akurasi dengan k=17 adalah 0.5
Akurasi dengan k=18 adalah 0.5053571428571428
Akurasi dengan k=19 adalah 0.49464285714285716
Akurasi dengan k=20 adalah 0.4928571428571429
Akurasi dengan k=21 adalah 0.49464285714285716
Akurasi dengan k=22 adalah 0.49464285714285716
Akurasi dengan k=23 adalah 0.5035714285714286
Akurasi dengan k=24 a

# ----------------------------------------------------------------------------------------

# ini beda metode klasifikasi

# -----------------------------------------------------------------------------------------

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [23]:
best_accuracy = 0
best_kernel = None

# Daftar kernel yang ingin diuji
kernels_to_test = ['linear', 'poly', 'rbf', 'sigmoid']

# Perulangan untuk mencari kernel terbaik
for kernel in kernels_to_test:
    svm = SVC(kernel=kernel)
    svm.fit(X_train_scaled, y_train)
    y_pred = svm.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Akurasi dengan kernel {kernel} adalah {accuracy}')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_kernel = kernel

print(f'Akurasi terbaik adalah {best_accuracy} dengan kernel {best_kernel}')

Akurasi dengan kernel linear adalah 0.525
Akurasi dengan kernel poly adalah 0.4303571428571429
Akurasi dengan kernel rbf adalah 0.5285714285714286
Akurasi dengan kernel sigmoid adalah 0.21071428571428572
Akurasi terbaik adalah 0.5285714285714286 dengan kernel rbf


In [24]:
knn = KNeighborsClassifier(n_neighbors= 25, metric = "euclidean")
dump(knn, open('modelknn5_fitur.pkl', 'wb'))


In [25]:
import pickle
with open('modelknn5_fitur.pkl', 'rb') as knn:
    loadknn= pickle.load(knn)
loadknn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=25)

In [26]:
y_pred = loadknn.predict(X_test_scaled)
y_pred

array(['OAF_neutral', 'YAF_happy', 'YAF_fear', 'YAF_disgust', 'OAF_Sad',
       'OAF_happy', 'YAF_disgust', 'OAF_angry', 'OAF_disgust', 'OAF_Sad',
       'YAF_sad', 'YAF_sad', 'OAF_angry', 'OAF_angry',
       'OAF_Pleasant_surprise', 'OAF_angry', 'OAF_Sad', 'YAF_disgust',
       'OAF_angry', 'YAF_neutral', 'YAF_happy', 'OAF_neutral',
       'OAF_angry', 'YAF_angry', 'OAF_Sad', 'YAF_sad',
       'YAF_pleasant_surprised', 'YAF_fear', 'OAF_happy', 'OAF_neutral',
       'OAF_Sad', 'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_angry',
       'YAF_fear', 'YAF_pleasant_surprised', 'OAF_Pleasant_surprise',
       'OAF_Sad', 'YAF_disgust', 'OAF_Fear', 'YAF_disgust', 'OAF_Sad',
       'YAF_neutral', 'YAF_disgust', 'OAF_Fear', 'OAF_angry', 'YAF_happy',
       'YAF_disgust', 'OAF_neutral', 'OAF_Sad', 'OAF_happy',
       'OAF_disgust', 'OAF_Sad', 'YAF_neutral', 'YAF_disgust', 'OAF_Sad',
       'OAF_angry', 'OAF_Pleasant_surprise', 'YAF_sad', 'OAF_Fear',
       'OAF_disgust', 'OAF_disgust', 'OAF_neut

In [27]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.5160714285714286


In [28]:
import pickle
from sklearn.decomposition import PCA

sklearn_pca = PCA(n_components=2)


In [29]:
with open('PCA1.pkl', 'wb') as pc:
    pickle.dump(sklearn_pca, pc)
